# Workflow: Caudoviricetes phylogeny 

Workflow for inference of Caudoviricetes phylogeny via concatenated protein alignments of putative single copy core genes

Based on the method of Low *et al.* (2019) [here](https://doi.org/10.1038/s41564-019-0448-z)

Method: 

- Previous analyses (Low *et al.* 2019) used 2017 version of VOG and the identified IDs are no longer compatible with the latest version
- in brief, the workflow below: 
  - re-identifies putative single copy core genes in Caudoviricetes viruses broadly following the method of Low *et al.* (2019), using latest viralRefSeq references and the latest VOG database
  - extracts the identified putative core genes from viralRefSeq references and Waiwera vOTUs
  - generates filtered concatenated core gene protein alignments for all Caudoviricetes viruses
  - build trees to infer phlyogeny and for visualisation of in iTol


***

## Reference databases

#### VOG database

In [ ]:
mkdir -p vogdb_v222_2024_04_10

wget -r --no-parent https://fileshare.csb.univie.ac.at/vog/latest/ -P vogdb_v222_2024_04_10/
mv vogdb_v222_2024_04_10/fileshare.csb.univie.ac.at/vog/latest/* vogdb_v222_2024_04_10/
rm -r vogdb_v222_2024_04_10/fileshare.csb.univie.ac.at/

#untar hmm profiles file and cat for downstream use
cd vogdb_v222_2024_04_10
tar -xzf vog.hmm.tar.gz
cat hmm/*.hmm > vogdb_all_hmm.hmm

tar -xzf vog.faa.tar.gz
gunzip vogdb.proteins.all.fa.gz

#### viralRefSeq references

Download latest database

In [ ]:
mkdir -p viralRefSeq_2024_04_10

wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz -P viralRefSeq_2024_04_10/
wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.faa.gz -P viralRefSeq_2024_04_10/
wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.genomic.gbff.gz -P viralRefSeq_2024_04_10/
wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.protein.gpff.gz  -P viralRefSeq_2024_04_10/

gunzip viralRefSeq_2024_04_10/*.gz


Filter RefSeq by taxonomy (Caudoviricetes)

In [ ]:
cd viralRefSeq_2024_04_10

module purge
module load Python/3.11.3-gimkl-2022a
python3

import pandas as pd
import numpy as np
import re
import glob
import os
from Bio import SeqIO

### Genomic files
metadata_dfs = []
with open('viral.1.genomic.gbff', 'r') as genbank_infile:
    with open('Caudoviricetes.genomic.fna', 'w') as write_fasta:
        for seq_record in SeqIO.parse(genbank_infile, "genbank") :
            if 'Caudoviricetes' in seq_record.annotations['taxonomy']:
                write_fasta.write(">%s\n%s\n" % (
                    seq_record.id,
                    str(seq_record.seq)))
                try:
                    host = seq_record.features[0].qualifiers['host'][0]
                except Exception as e:
                  host = np.nan
                metadata_dfs.append(pd.DataFrame({'genomeID': seq_record.id, 'full_taxonomy': ';'.join(seq_record.annotations['taxonomy']), 'host': host}, index=[0]))

df = pd.concat(metadata_dfs).reset_index(drop=True)

# separate taxonomy into ranks based on suffix extraction
df['domain'] = np.where(df['full_taxonomy'].astype(str).str.contains('(Viruses).*'), df['full_taxonomy'].astype(str).str.replace(pat='(Viruses).*', repl='\\1', regex=True), 'Unclassified')
tax_rank = ['realm', 'subrealm', 'kingdom', 'subkingdom', 'phylum', 'subphylum', 'class', 'subclass', 'order', 'suborder', 'family', 'subfamily', 'genus']
suffix_str = ['viria', 'vira', 'virae', 'virites', 'viricota', 'viricotina', 'viricetes', 'viricetidae', 'virales', 'virineae', 'viridae', 'virinae', 'virus']
for i in range(len(tax_rank)):
    df[tax_rank[i]] = np.where(df['full_taxonomy'].astype(str).str.contains('.*;(.*'+suffix_str[i]+').*'), df['full_taxonomy'].astype(str).str.replace(pat='.*;(.*'+suffix_str[i]+').*', repl='\\1', regex=True), 'Unclassified')

df['species'] = np.where(df['full_taxonomy'].astype(str).str.contains('.*;(.*virus .*)'), df['full_taxonomy'].astype(str).str.replace(pat='.*;(.*virus .*)', repl='\\1', regex=True), 'Unclassified')

# write out metadata file
df.to_csv('Caudoviricetes.genomic.metadata.tsv', sep='\t', index=False)

quit()


Run CheckV to assess predicted completeness

In [ ]:
module load CheckV/0.7.0-gimkl-2020a-Python-3.8.2

mkdir -p checkv_out

checkv end_to_end viralRefSeq_2024_04_10/Caudoviricetes.genomic.fna checkv_out -t 16 --quiet

*Slurm runtime < 40 hr; MaxRSS < 8 GB*

***

## Run DRAMv on Caudoviricetes references

#### DRAM-v Prep: Split fasta file

In [ ]:
mkdir -p RefSeq/DRAMv/vsort2_prepfiles/split_input_fasta

module load BBMap/38.95-gimkl-2020a

partition.sh \
in=viralRefSeq_2024_04_10/Caudoviricetes.genomic.fna \
out=RefSeq/DRAMv/vsort2_prepfiles/split_input_fasta/refseq_caudoviricetes_%.fna ways=100


#### DRAM-v Prep: VirSorter2

Note: run as a slurm array (`#SBATCH --array=0-99`)

In [ ]:
module load VirSorter/2.2.3-gimkl-2020a-Python-3.8.2

# Set up working directories
mkdir -p RefSeq/DRAMv/vsort2_prepfiles/subsets

# run virsorter2
virsorter run -j 24 \
--seqname-suffix-off --viral-gene-enrich-off --provirus-off --prep-for-dramv \
-i RefSeq/DRAMv/vsort2_prepfiles/split_input_fasta/refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID}.fna \
-d Databases/virsorter2_20210909/ \
--min-score 0 --include-groups dsDNAphage,NCLDV,RNA,ssDNA,lavidaviridae \
-l refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID} \
-w RefSeq/DRAMv/vsort2_prepfiles/subsets/refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID} \
--tmpdir ${SLURM_JOB_ID}.tmp \
--rm-tmpdir \
all \
--config LOCAL_SCRATCH=${TMPDIR:-/tmp}


*Slurm runtime < 40 min; MaxRSS < 1 GB*

#### DRAM-v: annotate

Note: run as a slurm array (`#SBATCH --array=0-99%20`)



In [ ]:
mkdir -p RefSeq/DRAMv/dramv_annotation

# Load DRAM (conda install)
module load Miniconda3/4.12.0
export CONDA_PKGS_DIRS=.conda/pkgs
export CONDA_ENVS_PATH=.conda/envs

# Run DRAM
source activate DRAM_1.4.6
DRAM-v.py annotate --threads 32 \
--min_contig_size 1000 \
-i RefSeq/DRAMv/vsort2_prepfiles/subsets/refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID}/refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID}-for-dramv/final-viral-combined-for-dramv.fa \
-v RefSeq/DRAMv/vsort2_prepfiles/subsets/refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID}/refseq_caudoviricetes_${SLURM_ARRAY_TASK_ID}-for-dramv/viral-affi-contigs-for-dramv.tab \
-o RefSeq/DRAMv/dramv_annotation/dramv_annotation_subset_${SLURM_ARRAY_TASK_ID}

conda deactivate


*Slurm runtime < 20 hr; MaxRSS < 16 GB*

#### DRAM-v: Compile DRAM-v annotation subsets and genes files

In [ ]:
# Working directory
cd /nesi/nobackup/ga02676/Waiwera_project/mikeh_Vir/MetaG/7b.Caudoviricetes/core_genes/RefSeq/DRAMv

# Load python
module purge
module load Python/3.8.2-gimkl-2020a

# Run compile dram annotations
compile_dram_annotations.py \
-i RefSeq/DRAMv/dramv_annotation \
-o RefSeq/DRAMv/dramv_annotation/collated_dramv_

# cat genes files
cat RefSeq/DRAMv/dramv_annotation/*/genes.faa > RefSeq/DRAMv/collated_dram_genes.faa
cat RefSeq/DRAMv/dramv_annotation/*/genes.fna > RefSeq/DRAMv/collated_dram_genes.fna


***

## HMM search of VOGdb HMMs against RefSeq references

- n.b. using downloaded VOGdb file vog.hmm.tar.gz (Compressed archive of the HMMER3 compatible Hidden Markov Models obtained from the multiple sequence alignments for each VOG)
- running with the genes.faa file output from DRAMv

In [ ]:
mkdir -p RefSeq/VOGdb_hmmsearch

module load HMMER/3.3.2-GCC-11.3.0

hmmsearch -E 1e-3 --cpu 24 \
--tblout RefSeq/VOGdb_hmmsearch/viralRefSeq.all_genes.vogdb \
--domtblout RefSeq/VOGdb_hmmsearch/viralRefSeq.all_genes.domain_hits.vogdb \
vogdb_v222_2024_04_10/vogdb_all_hmm.hmm \
RefSeq/DRAMv/collated_dram_genes.faa > /dev/null


*Slurm runtime < 16 hr; MaxRSS < 8 GB*

## HMM search of VOGdb HMMs against vOTUs

- running with the genes.faa file output from DRAMv

In [ ]:
mkdir -p vOTUs/VOGdb_hmmsearch

module load HMMER/3.3.2-GCC-11.3.0

srun hmmsearch -E 1e-3 --cpu 24 \
--tblout vOTUs/VOGdb_hmmsearch/votus.vogdb \
--domtblout vOTUs/VOGdb_hmmsearch/votus.domain_hits.vogdb \
vogdb_v222_2024_04_10/vogdb_all_hmm.hmm \
vOTUs/dramv_annotation.genes.faa > /dev/null


*Slurm runtime < 12 hr; MaxRSS < 4GB*


***

## Assess Caudoviricetes viralRefSeq VOGdb gene hits for putative single copy core genes

Method:

- Filter references for >= 95% completeness (predicted via CheckV)
- Select markers based on the following criteria: 
  - (1) present in ≥10% of bacterial and archaeal virus genomes
  - (2) average copy number ≤1.2
  - (3) average protein length >100 amino acid residues. 

In [ ]:
cd RefSeq/VOGdb_hmmsearch

module purge
module load Python/3.11.3-gimkl-2022a
python3

import pandas as pd
import numpy as np
import re
import os
from glob import glob
from Bio.SeqIO.FastaIO import SimpleFastaParser

df = pd.read_csv('viralRefSeq.all_genes.vogdb', comment='#', header=None, delimiter=r"\s+", usecols=[*range(0,18)]).reset_index(drop=True)
df.columns = ['target_name','accession','query_name','accession','fullSeq_eval','fullSeq_score','fullSeq_bias','bestDomain_eval','bestDomain_score','bestDomain_bias','exp','reg','clu','ov','env','dom','rep','inc']     
df['genomeID'] = df['target_name'].str.replace('-cat_.*', '', regex=True)

# Only keep match with lowest full seq eval for each geneID
df = df.sort_values(by=["target_name", "fullSeq_eval"], ascending=[True, True])
df = df.groupby("target_name", as_index=False).first()

# filter for high-quality and/or complete genomes
checkv_df = pd.read_csv('../checkv_out/quality_summary.tsv', delimiter="\t")
checkv_df = checkv_df[checkv_df['completeness'] >= 95].reset_index(drop=True)
df = df[df['genomeID'].isin(checkv_df['contig_id'].values)]

# Write out
df.to_csv('Refseq_vogdb_hmmsearch.compiled.tsv', sep='\t', index=False)

# VOGdb hits per genome
vog_counts_df = df.groupby('genomeID')["query_name"].apply(lambda x: x.groupby(x).size()).reset_index().pivot(index='genomeID', columns='level_1', values='query_name').reset_index()
# Select vog hits based on criteria listed above
## present in more than 10% of genomes
vog_counts_df = vog_counts_df.dropna(thresh=int(len(vog_counts_df)*0.1), axis=1)
## average copy number ≤1.2
cols_mean = vog_counts_df.mean(axis=0, numeric_only=True)
drop_cols = cols_mean[cols_mean >= 1.2].index
vog_counts_df.drop(columns=drop_cols, inplace=True)
vogIDs_to_keep = [col for col in vog_counts_df.columns if col != 'genomeID']
## average protein length >100 amino acid residues
seq_files = sorted(glob('../DRAMv/dramv_annotation/dramv_annotation_subset_*/genes.faa'))
seq_dict = {}
for seq_file in seq_files:
    with open(seq_file, 'r') as read_fasta:
        for name, seq in SimpleFastaParser(read_fasta):
            name_trim = re.sub(r' .*', r'', name)
            seq_dict[name_trim] = len(seq)

genes_df = df.merge(pd.DataFrame(seq_dict.items(), columns=['geneID','aa_seq_length']), how='left', left_on='target_name', right_on='geneID')
genes_df = genes_df[genes_df['query_name'].isin(vogIDs_to_keep)]
mean_seq_len_df = genes_df.groupby('query_name', as_index=False)['aa_seq_length'].mean()

# final filtered list of vogIDs
final_vogIDs = mean_seq_len_df[mean_seq_len_df['aa_seq_length'] >= 100]['query_name'].values.tolist()
# counts table of vogs per genome
vog_counts_final_df = vog_counts_df[['genomeID']+[col for col in vog_counts_df.columns if col in final_vogIDs]]
# write out
vog_counts_final_df.to_csv('Refseq_vogdb.core_genes.countTable.tsv', sep='\t', index=False)

# compile table of final vogIDs w/annotations
vog_annot = pd.read_csv('vogdb_v222_2024_04_10/vog.annotations.tsv', delimiter="\t")
vog_annot = vog_annot[vog_annot['#GroupName'].isin(final_vogIDs)].reset_index(drop=True).rename(columns={'#GroupName': 'vogdb_id'})
vog_annot = vog_annot[['vogdb_id', 'FunctionalCategory', 'ConsensusFunctionalDescription']]
# write out
vog_annot.to_csv('Refseq_vogdb.core_genes.annotations.tsv', sep='\t', index=False)

quit()


***

## Collate core genes for viralRefSeq references and vOTUs

- Method:
  - filter datasets to retain only genomes with predicted completeness >= 85%
  - extract gene matches based on vogdb_id core genes identified above
  - for each gene: write out faa file of sequences to align (and then concatenate after alignment)

In [ ]:
mkdir -p faa_files/hmmsearch_results_85

module purge
module load Python/3.9.9-gimkl-2020a
python3

import pandas as pd
import numpy as np
import re
from Bio.SeqIO.FastaIO import SimpleFastaParser

# marker genes
marker_genes_df = pd.read_csv("RefSeq/VOGdb_hmmsearch/Refseq_vogdb.core_genes.annotations.tsv", sep='\t').drop('FunctionalCategory', axis=1)
marker_genes_dict = dict(marker_genes_df.values)

### RefSeq
vog_df = pd.read_csv('RefSeq/VOGdb_hmmsearch/viralRefSeq.all_genes.vogdb', comment='#', header=None, delimiter=r"\s+", usecols=[*range(0,18)]).reset_index(drop=True)
vog_df.columns = ['target_name','accession','query_name','accession','fullSeq_eval','fullSeq_score','fullSeq_bias','bestDomain_eval','bestDomain_score','bestDomain_bias','exp','reg','clu','ov','env','dom','rep','inc']     
vog_df['genomeID'] = vog_df['target_name'].str.replace('-cat_.*', '', regex=True)
vog_df['target_name'] = vog_df['target_name'].str.replace('-cat_\d+', '', regex=True)
# Only keep match with lowest full seq eval for each geneID
vog_df = vog_df.sort_values(by=["target_name", "fullSeq_eval"], ascending=[True, True])
vog_df = vog_df.groupby("target_name", as_index=False).first()
# filter based on completeness
checkv_df = pd.read_csv('RefSeq/checkv_out/quality_summary.tsv', delimiter="\t")
checkv_df = checkv_df[checkv_df['completeness'] >= 85].reset_index(drop=True)
refseq_df = vog_df[vog_df['genomeID'].isin(checkv_df['contig_id'].values)]

### vOTUs
# Caudovirales vOTU IDs
tax_predict = pd.read_csv("vOTU.tax_predict_table.tsv", sep='\t')
caudo_votu_ids = tax_predict[tax_predict['Class_combined_prediction'] == 'Caudoviricetes']['Genome'].values.tolist()
# vog results
votu_df = pd.read_csv('vOTUs/VOGdb_hmmsearch/votus.vogdb', comment='#', header=None, delimiter=r"\s+", usecols=[*range(0,18)]).reset_index(drop=True)
votu_df.columns = ['target_name','accession','query_name','accession','fullSeq_eval','fullSeq_score','fullSeq_bias','bestDomain_eval','bestDomain_score','bestDomain_bias','exp','reg','clu','ov','env','dom','rep','inc']     
votu_df['genomeID'] = votu_df['target_name'].str.replace('-cat_.*', '', regex=True)
votu_df['target_name'] = votu_df['target_name'].str.replace('-cat_\d+', '', regex=True)
# Only keep match with lowest full seq eval for each geneID
votu_df = votu_df.sort_values(by=["target_name", "fullSeq_eval"], ascending=[True, True])
votu_df = votu_df.groupby("target_name", as_index=False).first()
# subset vOTUs_50 caudoviricetes
votu_df = votu_df[votu_df['genomeID'].isin(caudo_votu_ids)]
# filter based on completeness
checkv_df = pd.read_csv('checkv_vOTUs/vOTUs.quality_summary.tsv', delimiter="\t")
checkv_df = checkv_df[checkv_df['completeness'] >= 85].reset_index(drop=True)
votu_df = votu_df[votu_df['genomeID'].isin(checkv_df['contig_id'].values)]

### Extract marker genes, write sequences to new faa files
# faa files for: RefSeq references; Waiwera vOTUs; RefSeq references and vOTUs combined
for marker_gene in marker_genes_dict.keys():
    refseq_marker_gene_geneIDs = refseq_df[refseq_df['query_name'] == marker_gene].reset_index(drop=True)['target_name'].values.tolist()
    votu_marker_gene_geneIDs = votu_df[votu_df['query_name'] == marker_gene].reset_index(drop=True)['target_name'].values.tolist()
    # generate faa files of amino acid sequences
    with open('RefSeq/DRAMv/collated_dram_genes.faa', 'r') as read_fasta:
        with open('faa_files/hmmsearch_results_85/refseq.'+marker_gene+'.faa', 'w') as write_refseq_faa:
            with open('faa_files/hmmsearch_results_85/combined.'+marker_gene+'.faa', 'w') as write_combined_faa:
                for name, seq in SimpleFastaParser(read_fasta):
                    gene_id = re.sub(r'(.*)-cat_\d+(_\d*).*', r'\1\2', name)
                    if gene_id in refseq_marker_gene_geneIDs:
                        write_refseq_faa.write('>' + str(gene_id) + '_' + marker_gene + '\n' + str(seq) + '\n')
                        write_combined_faa.write('>' + str(gene_id) + '_' + marker_gene + '\n' + str(seq) + '\n')
    with open('vOTUs/dramv_annotation.genes.faa', 'r') as read_fasta:
        with open('faa_files/hmmsearch_results_85/votus.'+marker_gene+'.faa', 'w') as write_votu_faa:
            with open('faa_files/hmmsearch_results_85/combined.'+marker_gene+'.faa', 'a') as write_combined_faa:
                for name, seq in SimpleFastaParser(read_fasta):
                    gene_id = re.sub(r'(.*)-cat_\d+(_\d*).*', r'\1\2', name)
                    if gene_id in votu_marker_gene_geneIDs:
                        write_votu_faa.write('>' + str(gene_id) + '_' + marker_gene + '\n' + str(seq) + '\n')
                        write_combined_faa.write('>' + str(gene_id) + '_' + marker_gene + '\n' + str(seq) + '\n')


quit()


***

## Gene alignments for each marker gene

In [ ]:
#!/bin/bash
#SBATCH -A <>
#SBATCH -J caudo_core_genes_alignments_combined
#SBATCH --time 00:30:00
#SBATCH --mem=1GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH -e caudo_core_genes_alignments_combined_%a.err
#SBATCH -o caudo_core_genes_alignments_combined_%a.out

mkdir -p alignments/hmmsearch_results_85

file_list=( $(ls faa_files/hmmsearch_results_85/combined.*) )
file_id=${file_list[${SLURM_ARRAY_TASK_ID}]}
out_id=$(basename ${file_id} .faa)

module purge
module load Clustal-Omega/1.2.4-gimkl-2020a

clustalo -i ${file_id} -o alignments/hmmsearch_results_85/aln.${out_id}.faa --outfmt=fa --log=alignments/hmmsearch_results_85/aln.${out_id}.log --threads=16 --force


Run slurm array

In [ ]:
file_list=( $(ls faa_files/hmmsearch_results_85/combined.*) )
ZNUM=$(( ${#file_list[@]} - 1 ))

sbatch --array=0-${ZNUM} caudo_core_genes_alignments_combined.sl


*Slurm runtime < 15 min; MaxRss < 1 GB*

***

## Concatenate protein alignments for all genomes

- Concantenate above alignments into a single file
- Filter based on criteria from Low *et al.* (2019)
  - NOTEs from Low *et al.* for these steps:
    - The length of all markers combined before trimming was 26,165 amino acids
    - All marker MSAs were individually trimmed by **removing columns represented in <50% of taxa**. 
    - The individual alignments were concatenated by **introducing gaps in positions where markers were absent from a genome**
    - The resulting concatenated MSA of 23,531 columns was further filtered to **remove 182 genomes with <5% amino acid representation of the total alignment length**, producing a final matrix of 1,803 genomes× 23,531 columns with **84% missing data**.
    - The CCP77 reference tree was inferred from the MSA using **IQ-TREE** version 1.5.5 under the **partitioned LG + GAMMA model** and **midpoint rooted**, followed by **100 nonparametric bootstrap replicates** to estimate branch support.


RefSeq and vOTUs combined:

In [ ]:
mkdir -p concat_alignments/hmmsearch_results_85

module purge
module load Python/3.9.9-gimkl-2020a
python3

import pandas as pd
import numpy as np
import re
from Bio.SeqIO.FastaIO import SimpleFastaParser

# vog IDs
marker_genes_df = pd.read_csv("RefSeq/VOGdb_hmmsearch/Refseq_vogdb.core_genes.annotations.tsv", sep='\t').drop('FunctionalCategory', axis=1)
marker_genes_dict = dict(marker_genes_df.values)

# establish df
concat_alignments_df = pd.DataFrame(columns=['genomeID'])

# loop throgh each alignment, create as df, join with running full df
for vog_id in marker_genes_dict.keys():
    aa_seqs_dict = {}
    with open('alignments/hmmsearch_results_85/aln.combined.'+vog_id+'.faa', 'r') as read_fasta:
        for name, seq in SimpleFastaParser(read_fasta):
            aa_seqs_dict[name] = seq
    aa_seqs_df = pd.DataFrame(aa_seqs_dict.items(), columns=['geneID', 'seq_alignment'])
    aa_seqs_df['genomeID'] = aa_seqs_df['geneID'].str.replace('_\d+_VOG.*', '', regex=True)
    # if more than one gene for this vog in a genome, just take the first alignment
    aa_seqs_df = aa_seqs_df.drop_duplicates(subset='genomeID', keep="first")
    # split alignment into columns
    aa_seqs_df = pd.concat([aa_seqs_df['genomeID'], aa_seqs_df['seq_alignment'].apply(lambda x: pd.Series(list(x)))], axis=1)
    aa_seqs_df = aa_seqs_df.rename(columns={c: str(vog_id)+'_pos_'+str(c) for c in aa_seqs_df.columns if c not in ['genomeID']})
    # replace all * with '-'
    aa_seqs_df = aa_seqs_df.replace('*', '-')
    ## remove columns present in less than 50% of genomes
    aa_seqs_df = aa_seqs_df.replace('-', np.nan)
    aa_seqs_df = aa_seqs_df.dropna(thresh=int(len(aa_seqs_df)*0.5), axis=1)
    concat_alignments_df = concat_alignments_df.merge(aa_seqs_df, how="outer", on='genomeID')

# sort by genomeID
concat_alignments_df = concat_alignments_df.sort_values('genomeID').reset_index(drop=True)
# calculate number of columns remaining (i.e. number of amino acids)
len(concat_alignments_df.columns)-1
#13415
# count genomes
genomes_count = len(concat_alignments_df)
print(genomes_count)
#5246
# remove genomes with <5% amino acid representation of the total alignment length
concat_alignments_df = concat_alignments_df.dropna(subset=[col for col in concat_alignments_df.columns if col != 'genomeID'], thresh=int((len(concat_alignments_df.columns)-1)*0.05), axis=0).reset_index(drop=True)
# count filtered genomes
genomes_filt_count = len(concat_alignments_df)
print(genomes_filt_count)
#4481
print(genomes_count - genomes_filt_count)
#765 'genomes' removed

# Calculate % missing data
concat_alignments_df[[col for col in concat_alignments_df.columns if col != 'genomeID']].isna().mean().mean()
# 0.8494473913670058 = 85% missing data

# replace all nan with '-'
concat_alignments_df = concat_alignments_df.replace(np.nan, '-')

# Write out concatenated alignment file
concat_alignments_df['seq_alignment'] = concat_alignments_df[[col for col in concat_alignments_df.columns if col != 'genomeID']].astype(str).agg(''.join, axis=1)
concat_alignments_df = concat_alignments_df[['genomeID', 'seq_alignment']]
concat_alignments_dict = dict(concat_alignments_df.values)
with open('concat_alignments/hmmsearch_results_85/combined.core_genes.concatenated_alignment.faa', 'w') as write_faa:
    for key,value in concat_alignments_dict.items():
        write_faa.write('>' + str(key) + '\n' + str(value) + '\n')

quit()


vOTUs only (filter out of combined alignments)

In [ ]:
mkdir -p concat_alignments/hmmsearch_results_85

module purge
module load Python/3.9.9-gimkl-2020a
python3

import pandas as pd
import numpy as np
import re
from Bio.SeqIO.FastaIO import SimpleFastaParser

# vog IDs
marker_genes_df = pd.read_csv("RefSeq/VOGdb_hmmsearch/Refseq_vogdb.core_genes.annotations.tsv", sep='\t').drop('FunctionalCategory', axis=1)
marker_genes_dict = dict(marker_genes_df.values)

# establish df
concat_alignments_df = pd.DataFrame(columns=['genomeID'])

# loop throgh each alignment, create as df, join with running full df
for vog_id in marker_genes_dict.keys():
    aa_seqs_dict = {}
    with open('alignments/hmmsearch_results_85/aln.combined.'+vog_id+'.faa', 'r') as read_fasta:
        for name, seq in SimpleFastaParser(read_fasta):
            if 'vOTU' in name:
                aa_seqs_dict[name] = seq
    aa_seqs_df = pd.DataFrame(aa_seqs_dict.items(), columns=['geneID', 'seq_alignment'])
    aa_seqs_df['genomeID'] = aa_seqs_df['geneID'].str.replace('_\d+_VOG.*', '', regex=True)
    # if more than one gene for this vog in a genome, just take the first alignment
    aa_seqs_df = aa_seqs_df.drop_duplicates(subset='genomeID', keep="first")
    # split alignment into columns
    aa_seqs_df = pd.concat([aa_seqs_df['genomeID'], aa_seqs_df['seq_alignment'].apply(lambda x: pd.Series(list(x)))], axis=1)
    aa_seqs_df = aa_seqs_df.rename(columns={c: str(vog_id)+'_pos_'+str(c) for c in aa_seqs_df.columns if c not in ['genomeID']})
    # replace all * with '-'
    aa_seqs_df = aa_seqs_df.replace('*', '-')
    ## remove columns present in less than 50% of genomes
    aa_seqs_df = aa_seqs_df.replace('-', np.nan)
    aa_seqs_df = aa_seqs_df.dropna(thresh=int(len(aa_seqs_df)*0.5), axis=1)
    concat_alignments_df = concat_alignments_df.merge(aa_seqs_df, how="outer", on='genomeID')

# sort by genomeID
concat_alignments_df = concat_alignments_df.sort_values('genomeID').reset_index(drop=True)
# calculate number of columns remaining (i.e. number of amino acids)
len(concat_alignments_df.columns)-1
#14705
# count genomes
genomes_count = len(concat_alignments_df)
print(genomes_count)
#290
# remove genomes with <5% amino acid representation of the total alignment length
concat_alignments_df = concat_alignments_df.dropna(subset=[col for col in concat_alignments_df.columns if col != 'genomeID'], thresh=int((len(concat_alignments_df.columns)-1)*0.05), axis=0).reset_index(drop=True)
# count filtered genomes
genomes_filt_count = len(concat_alignments_df)
print(genomes_filt_count)
#202
print(genomes_count - genomes_filt_count)
#88 'genomes' removed

# Calculate % missing data
concat_alignments_df[[col for col in concat_alignments_df.columns if col != 'genomeID']].isna().mean().mean()
#0.8852660070495145 = 89% missing data

# replace all nan with '-'
concat_alignments_df = concat_alignments_df.replace(np.nan, '-')

# Write out concatenated alignment file
concat_alignments_df['seq_alignment'] = concat_alignments_df[[col for col in concat_alignments_df.columns if col != 'genomeID']].astype(str).agg(''.join, axis=1)
concat_alignments_df = concat_alignments_df[['genomeID', 'seq_alignment']]
concat_alignments_dict = dict(concat_alignments_df.values)
with open('concat_alignments/hmmsearch_results_85/votus.core_genes.concatenated_alignment.faa', 'w') as write_faa:
    for key,value in concat_alignments_dict.items():
        write_faa.write('>' + str(key) + '\n' + str(value) + '\n')

quit()


***

## Build trees


#### RefSeq and vOTUS combined - ultrafast bootstrap

In [ ]:
#!/bin/bash
#SBATCH -A <>
#SBATCH -J caudo_core_genes_iqtree_combined_ultrafastboot_nmax1000
#SBATCH --time 7-00:00:00
#SBATCH --mem=120GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=32
#SBATCH -e caudo_core_genes_iqtree_combined_ultrafastboot_nmax1000.err
#SBATCH -o caudo_core_genes_iqtree_combined_ultrafastboot_nmax1000.out

# Set up directories
cd concat_alignments/hmmsearch_results_85

# Load module
module purge
module load IQ-TREE/2.2.2.2-gimpi-2022a

# Run IQ-tree
iqtree -T 32 -m TEST -B 1000 -s combined.core_genes.concatenated_alignment.faa


*Slurm runtime < 21 days ; MaxRss < 50 GB*

#### vOTUS only - ultrafast bootstrap

In [ ]:
#!/bin/bash
#SBATCH -A ga02676
#SBATCH -J caudo_core_genes_iqtree_votus_ultrafastboot_nmax1000
#SBATCH --time 7-00:00:00
#SBATCH --mem=8GB
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=32
#SBATCH -e caudo_core_genes_iqtree_votus_ultrafastboot_nmax1000.err
#SBATCH -o caudo_core_genes_iqtree_votus_ultrafastboot_nmax1000.out

# Set up directories
cd concat_alignments/hmmsearch_results_85

# Load module
module purge
module load IQ-TREE/2.2.2.2-gimpi-2022a

# Run IQ-tree
iqtree -T 32 -m TEST -B 1000 -s votus.core_genes.concatenated_alignment.faa


*Slurm runtime < 6 hr; MaxRss < 2 GB*

***

***